### AQC
AQC的核心思想，绝热量子演化: H(f(s)) = (1 − f(s))H0 + f(s)H1, 0 ≤ s ≤ 1.
H1 是给定的
这里H0 是一个用来实现初态制备的，
$$
H_0=\sigma_x \otimes Q_b=\left(\begin{array}{cc}
0 & Q_b \\
Q_b & 0
\end{array}\right)
$$
$Q_b=$ $I_N-|b\rangle\langle b|$. 
then $H_0$ is a Hermitian matrix and the null space of $H_0$ is $\operatorname{Null}\left(H_0\right)=\operatorname{span}\{|\widetilde{b}\rangle,|\bar{b}\rangle\}$. Here, $|\widetilde{b}\rangle=$ $|0, b\rangle:=(b, 0)^{\top},|\bar{b}\rangle=|1, b\rangle:=(0, b)^{\top}$. The dimension of $H_0$ is $2 N$ and one ancilla qubit is needed to enlarge the matrix block. We also define
$$
H_1=\sigma_{+} \otimes\left(A Q_b\right)+\sigma_{-} \otimes\left(Q_b A\right)=\left(\begin{array}{cc}
0 & A Q_b \\
Q_b A & 0
\end{array}\right) .
$$
Here, $\sigma_{ \pm}=\frac{1}{2}\left(\sigma_x \pm \mathrm{i} \sigma_y\right)$. Note that if $|x\rangle$ satisfies $A|x\rangle \propto|b\rangle$, we have $Q_b A|x\rangle=Q_b|b\rangle=0$. Then $\operatorname{Null}\left(H_1\right)=\operatorname{span}\{|\widetilde{x}\rangle,|\bar{b}\rangle\}$ with $|\widetilde{x}\rangle=|0, x\rangle$. Since $Q_b$ is a projection operator, the gap between 0 and the rest of the eigenvalues of $H_0$ is 1 . The gap between 0 and the rest of the eigenvalues of $H_1$ is bounded from below by $1 / \kappa$ (see Appendix A).
### GATE-BASED IMPLEMENTATION OF AQC
We briefly discuss how to implement $\mathrm{AQC}(\mathrm{p})$ and $\mathrm{AQC}(\mathrm{exp})$ on a gate-based quantum computer. Since $\left|\psi_T(s)\right\rangle=\mathcal{T} \exp \left(-\mathrm{i} T \int_0^s H\left(f\left(s^{\prime}\right)\right) d s^{\prime}\right)\left|\psi_T(0)\right\rangle$, where $\mathcal{T}$ is the time-ordering operator, it is sufficient to implement an efficient time-dependent Hamiltonian simulation of $H(f(s))$.

One straightforward approach is to use the Trotter splitting method. The lowest order approximation takes the form
$$
\begin{aligned}
& \mathcal{T} \exp \left(-\mathrm{i} T \int_0^s H\left(f\left(s^{\prime}\right)\right) \mathrm{d} s^{\prime}\right) \approx \prod_{m=1}^M \exp \left(-\mathrm{i} T h H\left(f\left(s_m\right)\right)\right) \\
& \approx \prod_{m=1}^M \exp \left(-\mathrm{i} T h\left(1-f\left(s_m\right)\right) H_0\right) \exp \left(-\mathrm{i} T h f\left(s_m\right) H_1\right),
\end{aligned}
$$
where $h=s / M, s_m=m h$. It is proved in [31] that the error of such an approximation is $O$ (poly $\left.(\log (N)) T^2 / M\right)$, which indicates that to achieve an $\epsilon$-approximation, it suffices to choose $M=$ $O\left(\operatorname{poly}(\log (N)) T^2 / \epsilon\right)$. 


In [104]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from qiskit import QuantumCircuit
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from qiskit import QuantumCircuit


def gray_code(b):
    '''Gray code of b.
    Args:
        b: int:
            binary integer
    Returns:
        Gray code of b.
    '''
    return b ^ (b >> 1)


def gray_permutation(a):
    '''Permute the vector a from binary to Gray code order.

    Args:
        a: vector
            1D NumPy array of size 2**n
    Returns:
        vector:
            Gray code permutation of a
    '''
    b = np.zeros(a.shape[0])
    for i in range(a.shape[0]):
        b[i] = a[gray_code(i)]
    return b


def sfwht(a):
    '''Scaled Fast Walsh-Hadamard transform of input vector a.

    Args:
        a: vector
            1D NumPy array of size 2**n.
    Returns:
        vector:
            Scaled Walsh-Hadamard transform of a.
    '''
    n = int(np.log2(a.shape[0]))
    for h in range(n):
        for i in range(0, a.shape[0], 2**(h+1)):
            for j in range(i, i+2**h):
                x = a[j]
                y = a[j + 2**h]
                a[j] = (x + y) / 2
                a[j + 2**h] = (x - y) / 2
    return a


def compute_control(i, n):
    '''Compute the control qubit index based on the index i and size n.'''
    if i == 4**n:
        return 1
    return 2*n - int(np.log2(gray_code(i-1) ^ gray_code(i)))


def compressed_uniform_rotation(a, ry=True):
    '''Compute a compressed uniform rotation circuit based on the thresholded
    vector a.

    Args:
        a: vector:
            A thresholded vector a a of dimension 2**n
        ry: bool
            uniform ry rotation if true, else uniform rz rotation
    Returns:
        circuit
            A qiskit circuit representing the compressed uniform rotation.
    '''
    n = int(np.log2(a.shape[0])/2)
    circ = QuantumCircuit(2*n + 1)

    i = 0
    while i < a.shape[0]:
        parity_check = 0

        # add the rotation gate
        if a[i] != 0:
            if ry:
                circ.ry(a[i], 0)
            else:
                circ.rz(a[i], 0)

        # loop over sequence of consecutive zeros
        while True:
            ctrl = compute_control(i+1, n)
            # toggle control bit
            parity_check = (parity_check ^ (1 << (ctrl-1)))
            i += 1
            if i >= a.shape[0] or a[i] != 0:
                break

        # add CNOT gates
        for j in range(1, 2*n+1):
            if parity_check & (1 << (j-1)):
                circ.cx(j, 0)

    return circ

def fable(a, epsilon=None):
    '''FABLE - Fast Approximate BLock Encodings.

    Args:
        a: array
            matrix to be block encoded.
        epsilon: float >= 0
            (optional) compression threshold.
    Returns:
        circuit: qiskit circuit
            circuit that block encodes A
        alpha: float
            subnormalization factor
    '''
    epsm = np.finfo(a.dtype).eps
    alpha = np.linalg.norm(np.ravel(a), np.inf)
    if alpha > 1:
        alpha = alpha + np.sqrt(epsm)
        a = a/alpha
    else:
        alpha = 1.0

    n, m = a.shape
    if n != m:
        k = max(n, m)
        a = np.pad(a, ((0, k - n), (0, k - m)))
        n = k
    logn = int(np.ceil(np.log2(n)))
    if n < 2**logn:
        a = np.pad(a, ((0, 2**logn - n), (0, 2**logn - n)))
        n = 2**logn

    a = np.ravel(a)

    if all(np.abs(np.imag(a)) < epsm):  # real data
        a = gray_permutation(
                sfwht(
                    2.0 * np.arccos(np.real(a))
                )
            )
        # threshold the vector
        if epsilon:
            a[abs(a) <= epsilon] = 0
        # compute circuit
        OA = compressed_uniform_rotation(a)
    else:  # complex data
        # magnitude
        a_m = gray_permutation(
                sfwht(
                    2.0 * np.arccos(np.abs(a))
                )
            )
        if epsilon:
            a_m[abs(a_m) <= epsilon] = 0

        # phase
        a_p = gray_permutation(
                sfwht(
                    -2.0 * np.angle(a)
                )
            )
        if epsilon:
            a_p[abs(a_p) <= epsilon] = 0

        # compute circuit
        OA = compressed_uniform_rotation(a_m).compose(
                compressed_uniform_rotation(a_p, ry=False)
            )

    circ = QuantumCircuit(2*logn + 1)

    # diffusion on row indices
    for i in range(logn):
        circ.h(i+1)

    # matrix oracle
    circ = circ.compose(OA)

    # swap register
    for i in range(logn):
        circ.swap(i+1,  i+logn+1)

    # diffusion on row indices
    for i in range(logn):
        circ.h(i+1)

    # reverse bits because of little-endiannes
    circ = circ.reverse_bits()

    return circ, alpha

In [183]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from scipy.linalg import expm

# Define parameters
 # 
T = 1.0  # total evolution time
M = 200  # number of Trotter steps
epsilon = 0.01  # error tolerance
h = T / M  # time step
n = 1
# Example matrices A and b (replace with actual matrices)
A = np.array([[2, 1], [1, 0]])
b = np.array([3, 1])
b = b / np.linalg.norm(b)  # Normalize vector b
N = 2*n
# Define Hamiltonians H0 and H1
def get_H0(N, b):
    Q_b = np.eye(N) - np.outer(b, b.conj())
    H0 = np.kron(np.array([[0, 1], [1, 0]]), Q_b)
    return H0

def get_H1(A, N, b):
    Q_b = np.eye(N) - np.outer(b, b.conj())
    H1 = np.kron(np.array([[0, 1], [0, 0]]), A @ Q_b) + np.kron(np.array([[0, 0], [1, 0]]), Q_b @ A)
    return H1

# Define the initial state |psi_T(0)>
def initial_state(N,b):
    qc = QuantumCircuit(5,5)  # N system qubits + 1 ancilla
    qc.initialize(b,1)
    return qc

# Implement the Trotterized evolution
def trotter_step(qc, H0, H1, f, dt):
    U0 = np.eye(H1.shape[0])- 1j  * (1-f) * H0
    U1 = np.eye(H1.shape[0])- 1j  * f * H1
    circH0,alpha = fable(U0)
    # circH1.measure(range(N,circH1.num_qubits),range(N,circH1.num_qubits))
    qc.append(circH0, range(circH0.num_qubits))
    circH1,alpha = fable(U1)
    # circH1.measure(range(N,circH1.num_qubits),range(N,circH1.num_qubits))
    qc.append(circH1, range(circH1.num_qubits))
    # qc.measure(range(N,circH1.num_qubits),range(N,circH1.num_qubits))

# Main function to simulate AQC
def simulate_AQC(N, M, H0, H1,b):
    f = lambda s: s
    qc = initial_state(N,b)
    
    for m in range(M):
        s = (m + 1) * h
        f_s = f(s)
        trotter_step(qc, H0, H1, f_s, h)
    # qc.measure(range(N,5),range(N,5))
    return qc

# Get Hamiltonians H0 and H1
H0 = get_H0(N, b)
H1 = get_H1(A, N, b)
# Simulate the AQC
final_qc = simulate_AQC(N, M, H0, H1,b)

# Run the simulation
# final_qc.draw()


In [184]:
## excute the circuit and get the statevector
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram

# execute the circuit on a simulator
backend = Aer.get_backend('statevector_simulator')
job = execute(final_qc, backend)
result = job.result()

# get the statevector
np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
statevector = result.get_statevector()
print(statevector)


Statevector([ 0.474-0.01j ,  0.   +0.j   ,  0.158-0.063j,  0.   +0.j   ,
              0.   -0.462j,  0.   +0.j   ,  0.   -0.094j,  0.   +0.j   ,
              0.474+0.01j ,  0.   +0.j   , -0.158-0.063j,  0.   +0.j   ,
             -0.   +0.462j,  0.   +0.j   ,  0.   -0.094j,  0.   +0.j   ,
             -0.   +0.047j,  0.   +0.j   , -0.   +0.07j ,  0.   +0.j   ,
              0.   -0.095j,  0.   +0.j   ,  0.   -0.085j,  0.   +0.j   ,
             -0.   -0.047j,  0.   +0.j   ,  0.   +0.07j ,  0.   +0.j   ,
             -0.   +0.095j,  0.   +0.j   ,  0.   -0.085j,  0.   +0.j   ],
            dims=(2, 2, 2, 2, 2))


## 全是unitary 的测试版本

In [179]:
import numpy as np
import pennylane as qml
from scipy.linalg import expm


# Define Hamiltonians H0 and H1
def get_H0(N, b):
    Q_b = np.eye(N) - np.outer(b, b.conj())
    H0 = np.kron(np.array([[0, 1], [1, 0]]), Q_b)
    return H0

def get_H1(A, N, b):
    Q_b = np.eye(N) - np.outer(b, b.conj())
    H1 = np.kron(np.array([[0, 1], [0, 0]]), A @ Q_b) + np.kron(np.array([[0, 0], [1, 0]]), Q_b @ A)
    return H1

# Define the initial state |psi_T(0)>
def initial_state(N,b):
    qml.QubitStateVector(b, wires=1)
    # qml.Hadamard(1)
# Implement the Trotterized evolution
def trotter_step( H0, H1, f, h):
    # print(f"{f},{h}")
    U0 = expm(-1j * (1 - f) * H0)
    U1 = expm(-1j * f * H1)
    # Apply U0 and U1 to the circuit
    qml.QubitUnitary(U0, range(n))
    qml.QubitUnitary(U1, range(n))

# Main function to simulate AQC
@qml.qnode(qml.device("default.qubit", wires=n))
def simulate_AQC(N, M, A, b):
    H0 = get_H0(N, b)
    H1 = get_H1(A, N, b)
    initial_state(N,b)
    for m in range(M):
        s = (m + 1) * h
        trotter_step(H0, H1, s, h)
    return qml.state()
# Define parameters
T = 1  # total evolution time
M = 1000  # number of Trotter steps
h = T / M  # time step
# Example matrices A and b (replace with actual matrices)
A = np.array([[2, 1], [1, 0]])
b = np.array([3, 1])

N = A.shape[0]  # number of A dimensions
b = b / np.linalg.norm(b)  # Normalize vector b
n = int(np.log2(2*N))  # number of qubits
# Simulate AQC
psi_T = simulate_AQC(N, M, A, b)
# Compute the overlap with the target state |psi_T>
overlap = np.abs(np.vdot(psi_T[:2],np.array([1,1]))) ** 2
print("Overlap with target state: ", overlap)


Overlap with target state:  1.999989204202271


## 只用了numpy库，没有用到其他库

In [48]:
import numpy as np
from scipy.linalg import expm


# Define Hamiltonians H0 and H1
def get_H0(N, b):
    Q_b = np.eye(N) - np.outer(b, b.conj())
    H0 = np.kron(np.array([[0, 1], [1, 0]]), Q_b)
    return H0

def get_H1(A, N, b):
    Q_b = np.eye(N) - np.outer(b, b.conj())
    H1 = np.kron(np.array([[0, 1], [0, 0]]), A @ Q_b) + np.kron(np.array([[0, 0], [1, 0]]), Q_b @ A)
    return H1

# Define the initial state |psi_T(0)>
def initial_state(N,b):
    state = np.kron(np.array([1, 0]), b)
    return state
    # qml.Hadamard(1)
# Implement the Trotterized evolution
def trotter_step(state, H0, H1, f, h):
    # print(f"{f},{h}")
    U0 = expm(-1j * (1 - f) * H0)
    # U0 = np.eye(2**n)- 1j*(1-f)*H0*h
    # U0 = U0/np.trace(U0@U0.conj().T)*4
    # U1 = np.eye(2**n)- 1j*f*H1*h
    U1 = expm(-1j * f * H1)
    # U1 = U1/np.trace(U0@U0.conj().T)*4
    # Apply U0 and U1 to the circuit
    return U1 @ U0 @ state

def simulate_AQC(N, M, A, b):
    H0 = get_H0(N, b)
    H1 = get_H1(A, N, b)
    state = initial_state(N,b)
    for m in range(M):
        s = (m + 1) * h
        state = trotter_step(state,H0, H1, s, h)
    return state
# Define parameters
T = 1  # total evolution time
M = 400  # number of Trotter steps
h = T / M  # time step
# Example matrices A and b (replace with actual matrices)
A = np.array([[2, 1], [1, 0]])
b = np.array([3, 1])

N = A.shape[0]  # number of A dimensions
b = b / np.linalg.norm(b)  # Normalize vector b
n = int(np.log2(2*N))  # number of qubits
# Simulate AQC
psi_T = simulate_AQC(N, M, A, b)
# Compute the overlap with the target state |psi_T>
overlap = np.abs(np.vdot(psi_T[:2],np.array([1,1])/np.sqrt(2))) ** 2
print("Overlap with target state: ", overlap)


Overlap with target state:  0.9992131923275845


In [49]:
simulate_AQC(N,M,A,b),b

(array([-0.69916157+0.j        , -0.71449552+0.j        ,
         0.        +0.00818073j,  0.        -0.02454218j]),
 array([0.9486833 , 0.31622777]))

In [33]:
Q_b = np.eye(N) - np.outer(b, b.conj())

In [34]:
U0 = np.eye(2**n)- 1j*(1-0.01)*get_H0(N, b)

In [39]:
U0@U0.conj().T

array([[ 1.09801+0.j, -0.29403+0.j,  0.     +0.j,  0.     +0.j],
       [-0.29403+0.j,  1.88209+0.j,  0.     +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.     +0.j,  1.09801+0.j, -0.29403+0.j],
       [ 0.     +0.j,  0.     +0.j, -0.29403+0.j,  1.88209+0.j]])

In [41]:
np.trace(U0)

(4+0j)